# Orchestration and ML Pipelines

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=brightgreen)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/inefficient-networks/blob/master/docs/notebooks/mlops/03-prefect)
[![Stars](https://img.shields.io/github/stars/particle1331/inefficient-networks?style=social)](https://github.com/particle1331/inefficient-networks)

```text
𝗔𝘂𝘁𝗵𝗼𝗿: Ron Medina
𝗟𝗮𝘀𝘁 𝗠𝗼𝗱𝗶𝗳𝗶𝗲𝗱: 2022/06/29

𝗗𝗲𝘀𝗰𝗿𝗶𝗽𝘁𝗶𝗼𝗻:
• Workflow orchestration using Prefect 2.0
• Running MLflow deployment jobs with Prefect

𝗔𝘁𝘁𝗿𝗶𝗯𝘂𝘁𝗶𝗼𝗻: 
Notes for Module 3 of the MLOps Zoomcamp by DataTalks.Club
• https://www.prefect.io/opensource/v2/
• https://datatalks.club/
• https://github.com/DataTalksClub/mlops-zoomcamp/
```

---

## Introduction

In the previous module, we learned about experiment tracking and model registry.
In particular, we discussed how to get a candidate model and promote it from staging to production.
In this module, we learn how to automate this process, and having this scheduled with workflow orchestration &mdash; specifically with [**Prefect 2.0**](https://orion-docs.prefect.io/).

Prefect allows us to programatically author, schedule, and monitor workflows. One of the primary goals of Prefect is to minimize time spent on **negative engineering**, i.e. coding against all possible causes of failure. As there are endless ways that elements of a data pipeline can fail, you can potentially spend an endless amount of time on negative engineering. In practical terms, Prefect provides tools such as retries, concurrency, logging, a nice UI, tracking dependencies, a database, caching and serialization, parameterization of scheduled tasks, and more. As we shall see later, this can add observability to every step in the data pipeline.

## Prefect flows

A **flow** in Prefect is simply a Python function. This consists of **tasks** which can be thought of as the atom of observability in Prefect. In practice, to create a flow, you simply convert functions that make it up into tasks. Consider the following example from the [*Getting Started with Prefect 2.0*](https://www.prefect.io/guide/blog/getting-started-prefect-2/#Makingourflowsbetterwithtasks) blog post. Here, we simulate getting data from an unreliable API, augmenting the fetched data, and writing the resulting data into a database. 

In [3]:
import random
from prefect import flow, task 


@task(retries=3)
def call_unreliable_api():
    choices = [{"data": 42}, "Failure"]
    res = random.choice(choices)
    if res == "Failure":
        raise Exception("Our unreliable service failed.")
    else:
        return res

@task
def augment_data(data: dict, msg: str):
    data["message"] = msg
    return data

@task
def write_to_database(data: dict):
    print(f"Wrote {data} to database successfully!")
    return "Success!"

@flow 
def pipeline(msg: str):
    api_result = call_unreliable_api()
    augmented_data = augment_data(data=api_result, msg=msg)
    write_to_database(augmented_data)


pipeline(0) # Augment data with zero.

17:26:54.662 | INFO    | prefect.engine - Created flow run 'juicy-platypus' for flow 'pipeline'
17:26:54.664 | INFO    | Flow run 'juicy-platypus' - Using task runner 'ConcurrentTaskRunner'
17:26:54.671 | WARNING | Flow run 'juicy-platypus' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
17:26:54.708 | INFO    | Flow run 'juicy-platypus' - Created task run 'call_unreliable_api-48f93715-0' for task 'call_unreliable_api'
17:26:54.729 | INFO    | Flow run 'juicy-platypus' - Created task run 'augment_data-505b3e0c-0' for task 'augment_data'
17:26:54.741 | ERROR   | Task run 'call_unreliable_api-48f93715-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/particle1331/miniforge3/envs/prefect/lib/python3.9/site-packages/prefect/engine.py", line 798, in orchestrate_task_run
    result = await run_sync_in_worker_thread(task.fn, *args, **kwargs)
  File "/Us

## Prefect Orion UI

Notice that this failed before pushing through. We can start the UI by calling `prefect orion start` in any directory (`.prefect` is saved in the system's root directory). This starts the Prefect Orion server in port 4200.

```bash
$ prefect orion start
Starting...

 ___ ___ ___ ___ ___ ___ _____    ___  ___ ___ ___  _  _
| _ \ _ \ __| __| __/ __|_   _|  / _ \| _ \_ _/ _ \| \| |
|  _/   / _|| _|| _| (__  | |   | (_) |   /| | (_) | .` |
|_| |_|_\___|_| |___\___| |_|    \___/|_|_\___\___/|_|\_|

Configure Prefect to communicate with the server with:

    prefect config set PREFECT_API_URL=http://127.0.0.1:4200/api

Check out the dashboard at http://127.0.0.1:4200



INFO:     Started server process [20557]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4200 (Press CTRL+C to quit)
```

We navigate around the UI to find the `pipeline` flow and its most recent which, as we have seen in the logs, was able to complete its execution. Here we see that this flow started on `2022/06/10 11:05:51 PM` and ended on `2022/06/10 11:05:52 PM`. We also see the logs has the details of the exception when the API call failed. In the second tab, we can see the tasks that make up this flow. There is also the subflow tab which shows that we can call flows from a parent flow.

```{figure} ../../../img/hello-world-2.png
---
---
```

One of the more interesting features of the dashboard is **Radar** on the right. This shows the dependence between tasks. Notice the linear dependence of the tasks, e.g. `write_to_database` depends on `augment_data` task but not on `call_unreliable_api`. Hovering on the tasks show the backward and forward data dependencies. Having tasks arranged in concentric circles allow for a heirarchy of dependence. Note that the runtime for each task is also conveniently displayed.

```{figure} ../../../img/hello-world-1.png
---
---
```

Finally, let us look at a flow which failed to complete all its tasks. Here all calls to the API failed despite the retries. The radar plot nicely shows where the flow has failed. This is really useful, especially when we have a dozens task and multiple subflows happening in our data pipeline.


```{figure} ../../../img/hello-world-3.png
```

<br>

**Remark.** Note also that geometrically there is more space available to grow the dependence tree compared to top-down or left-right approaches due to nodes being farther apart as we move radially with a fixed angle, this also allows Radar to minimize edge crossing by combining radial and circumferential movement for the edges between task nodes. 

Furthermore, Radar dynamically updates as tasks complete (or fails). The mini-map, edge tracing, and node selection tools make workflow inspection doable even for highly complex graphs. See [*Introducing Radar*](https://www.prefect.io/guide/blog/introducing-radar/) for further reading.



## MLflow runs as flow

In this section, we will write Prefect flows for running modelling experiments as a flow in Prefect. Our idea is to define a `main` flow which consists of one subflow `preprocess_data` and two tasks which will execute MLflow runs. Note that the output of the `preprocess_data` subflow will be used by the two MLflow runs, so there will be some data dependency. 

```{margin}
[`utils.py`](https://github.com/particle1331/inefficient-networks/blob/85993956250601edeccf0d1bd5f192bb20873677/docs/notebooks/mlops/3-prefect/utils.py)
```
```python
@task
def load_training_dataframe(file_path, y_min=1, y_max=60):
    """Load data from disk and preprocess for training."""
    
    # Load data from disk
    data = pd.read_parquet(file_path)

    # Create target column and filter outliers
    data['duration'] = data.lpep_dropoff_datetime - data.lpep_pickup_datetime
    data['duration'] = data.duration.dt.total_seconds() / 60
    data = data[(data.duration >= y_min) & (data.duration <= y_max)]

    return data


@task
def fit_preprocessor(train_data):
    """Fit and save preprocessing pipeline."""

    # Unpack passed data
    y_train = train_data.duration.values
    X_train = train_data.drop('duration', axis=1)    

    # Initialize pipeline
    categorical = ['PU_DO']
    numerical = ['trip_distance']

    preprocessor = make_pipeline(
        PrepareFeatures(categorical, numerical),
        DictVectorizer(),
    )

    # Fit only on train set
    preprocessor.fit(X_train, y_train)
    joblib.dump(preprocessor, artifacts / 'preprocessor.pkl')
    
    return preprocessor


@task
def create_model_features(preprocessor, train_data, valid_data):
    """Fit feature engineering pipeline. Transform training dataframes."""

    # Unpack passed data
    y_train = train_data.duration.values
    y_valid = valid_data.duration.values
    X_train = train_data.drop('duration', axis=1)
    X_valid = valid_data.drop('duration', axis=1)
    
    # Feature engineering
    X_train = preprocessor.transform(X_train)
    X_valid = preprocessor.transform(X_valid)

    return X_train, y_train, X_valid, y_valid


@flow
def preprocess_data(train_data_path, valid_data_path):
    """Return feature and target arrays from paths. 
    Note: This just combines all the functions above in a single step."""

    train_data = load_training_dataframe(train_data_path)
    valid_data = load_training_dataframe(valid_data_path)
    preprocessor = fit_preprocessor(train_data)

    # X_train, y_train, X_valid, y_valid
    return create_model_features(preprocessor, train_data, valid_data).result()
```

Next, we will create the `main` flow described above. Here we are passing around a [`PrefectFuture`](https://orion-docs.prefect.io/api-ref/prefect/futures/) object instead of Python objects. Futures represent the execution of a task and allow retrieval of the task run's state. This so that Prefect is able to track data dependency between tasks &mdash; converting to Python objects, i.e. using `.result()`, breaks this lineage. Note that once a future has been passed into the function, then we can treat it as a usual Python object as the `task` wrapper has done work to unpack the future object into Python objects.

For the `main` flow, we execute the following sequentially: running a subflow for preprocessing the datasets for modelling, training a linear regression baseline model, and training XGBoost models with different hyperparameters sampled using [TPE](https://optunity.readthedocs.io/en/latest/user/solvers/TPE.html). Sequential execution ensures that all resources are allocated to a single learning algorithm at each point in the flow run.

```{margin}
[`main.py`](https://github.com/particle1331/inefficient-networks/blob/85993956250601edeccf0d1bd5f192bb20873677/docs/notebooks/mlops/3-prefect/main.py)
```
```python
def objective(params, xgb_train, y_train, xgb_valid, y_valid):
    """Compute validation RMSE (one trial = one run)."""

    with mlflow.start_run():
        
        model = xgb.train(
            params=params,
            dtrain=xgb_train,
            num_boost_round=100,
            evals=[(xgb_valid, 'validation')],
            early_stopping_rounds=5,
            verbose_eval=False
        )

        # MLflow logging
        ...

    return {'loss': rmse_valid, 'status': STATUS_OK}


@task
def xgboost_runs(num_runs, data):
    """Run TPE algorithm on search space to minimize objective."""

    X_train, y_train, X_valid, y_valid = data
    xgb_train = xgb.DMatrix(X_train, label=y_train)
    xgb_valid = xgb.DMatrix(X_valid, label=y_valid)

    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }

    best_result = fmin(
        fn=partial(
            objective, 
            xgb_train=xgb_train, y_train=y_train, 
            xgb_valid=xgb_valid, y_valid=y_valid,
        ),
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_runs,
        trials=Trials()
    )


@task
def linreg_runs(data):
    """Run linear regression training."""

    X_train, y_train, X_valid, y_valid = data
    
    with mlflow.start_run():

        model = LinearRegression()
        model.fit(X_train, y_train)

        # MLflow logging
        ...

        
@flow(task_runner=SequentialTaskRunner())
def main(
    train_data_path, 
    valid_data_path, 
    num_xgb_runs, 
    experiment_name,
    tracking_uri,
):
    # Set and run experiment
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(experiment_name)

    data = preprocess_data(train_data_path, valid_data_path)
    linreg_runs(data)
    xgboost_runs(num_xgb_runs, data)
```


Looking at the dashboard, we can see a run of the `main` flow. As expected, this consists of 3 tasks that are executed sequentially as indicated in the timeline. 

```{figure} ../../../img/mlflow-runs-dashboard.png
---
---
```

Recall that the first task `preprocess-data` is a subflow. This has concurrent execution which we can see from overlapping lines in its timeline graph. This subflow consists of four tasks.

```{figure} ../../../img/radar_preprocessing.png
---
---
```

If we check out the radar of the `main` flow, we see the following. Here in an earlier screenshot, we see that `xgboost_runs` is currently running for 1 minute and 14 seconds. Note that both MLflow runs depending on the preprocessing subflow can be seen by the presence of edges. We can go down on the radar for the `preprocess-data` subflow by clicking on the `4 task runs` button.


```{figure} ../../../img/radar_xgb.png
---
---
```

Here we see the radar plot. Hovering on each task shows its data dependence on other tasks. For each task, the forward and backward data dependency edges are shown by the Radar graph. For example, in the figure below we can see the dependencies for the task `load_training_dataframe` that loads the train dataset. This sends data to the preprocessor (for training) and to the final task (for transformation) which returns all processed data for modelling, so we can see two forward dependencies.

```{figure} ../../../img/radar.png
---
---

```

## Deployment

In this section, we deploy a workflow that puts the best model in an MLflow experiment to staging in the model registry. This can be useful for regularly staging candidate models models trained on new data. The staged models can then be further checked if it should be deployed into production. The code below is covered in [Experiment Tracking and Model Management](https://particle1331.github.io/inefficient-networks/notebooks/mlops/2-mlflow/2-mlflow.html#api-workflows).

### Model staging review

Below we will run MLflow on localhost with SQLite. This can be easily adapted to a remote tracking server we only have to change `MLFLOW_TRACKING_URI`. Connecting to the client:

In [151]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


def print_experiment(experiment):
    print(f"(Experiment)")
    print(f"    experiment_id={experiment.experiment_id}")
    print(f"    name='{experiment.name}'")
    print(f"    artifact_location='{experiment.artifact_location}'")
    print()

for experiment in client.list_experiments():
    print_experiment(experiment)

(Experiment)
    experiment_id=0
    name='Default'
    artifact_location='./mlruns/0'

(Experiment)
    experiment_id=1
    name='nyc-taxi-experiment'
    artifact_location='./mlruns/1'



Recall that the flow performs training a linear model and XGBoost models with different parameters. As sort of minimum requirements, we only consider those models with validation RMSE less than `6.5` and inference time less than `2e-5`.

In [162]:
candidates = client.search_runs(
    experiment_ids=1,
    filter_string='metrics.rmse_valid < 6.5 and metrics.inference_time < 20e-6',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse_valid ASC"]
)

for run in candidates:
    print(f"run_id: {run.info.run_id}   rmse_valid: {run.data.metrics['rmse_valid']:.3f}   inference_time: {run.data.metrics['inference_time']:.4e}")


run_id: 34d34a6ba92a4494bc1e6a7604b303f2   rmse_valid: 6.427   inference_time: 7.6936e-06
run_id: fcf8eef67dd64db2848e0fecc1969914   rmse_valid: 6.481   inference_time: 4.2133e-06


Having satisfied the operating requirements, we can take the model with lowest error as the new version and set it up for staging. Note that this flow can fail when there are no experiments in our tracker. But that is okay as it serves as notification for us to look into our models.

In [164]:
model_to_stage = candidates[0]

registered_model = mlflow.register_model(
    model_uri=f"runs:/{model_to_stage.info.run_id}/model", 
    name='NYCRideDurationModel'
)

client.transition_model_version_stage(
    name='NYCRideDurationModel',
    version=registered_model.version, 
    stage='Staging',
)

Successfully registered model 'NYCRideDurationModel'.
2022/06/13 00:25:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: NYCRideDurationModel, version 1
Created version '1' of model 'NYCRideDurationModel'.


<ModelVersion: creation_timestamp=1655051149815, current_stage='Staging', description=None, last_updated_timestamp=1655051149828, name='NYCRideDurationModel', run_id='34d34a6ba92a4494bc1e6a7604b303f2', run_link=None, source='./mlruns/1/34d34a6ba92a4494bc1e6a7604b303f2/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

```{figure} ../../../img/mlflow-automatic-staging.png
---
---
Staged model from code cells above.
```

### MLflow staging flow

This looks good, so we collect the above code cells along into a workflow which will create a new experiment, perform the experiment runs, and filter the best model for staging. We will then schedule this workflow to be run at fixed intervals using Prefect.

```{margin}
[`main.py`](https://github.com/particle1331/inefficient-networks/blob/06a1ecfea5456430538d13cdebbbe4c23dbbe93c/docs/notebooks/mlops/3-prefect/main.py)
```
```python
@task
def stage_model(tracking_uri, experiment_name):
    """Register and stage best model."""

    # Get best model from current experiment
    client = MlflowClient(tracking_uri=tracking_uri)
    candidates = client.search_runs(
        experiment_ids=client.get_experiment_by_name(experiment_name).experiment_id,
        filter_string='metrics.rmse_valid < 6.5 and metrics.inference_time < 20e-6',
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=5,
        order_by=["metrics.rmse_valid ASC"]
    )

    # Register and stage best model
    best_model = candidates[0]
    registered_model = mlflow.register_model(
        model_uri=f"runs:/{best_model.info.run_id}/model", 
        name='NYCRideDurationModel'
    )

    client.transition_model_version_stage(
        name='NYCRideDurationModel',
        version=registered_model.version, 
        stage='Staging',
    )

    # Update description of staged model
    client.update_model_version(
        name='NYCRideDurationModel',
        version=registered_model.version,
        description=f"[{datetime.now()}] The model version {registered_model.version} from experiment '{experiment_name}' was transitioned to Staging."
    )

...

@flow(name='mlflow-staging', task_runner=SequentialTaskRunner())
def mlflow_staging(
    train_data_path, 
    valid_data_path, 
    num_xgb_runs=1
):    
    # Setup experiment
    ctx = get_run_context()
    MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
    EXPERIMENT_NAME = f"nyc-taxi-experiment-{ctx.flow_run.expected_start_time}"

    # Make experiment runs
    main(
        train_data_path=train_data_path, 
        valid_data_path=valid_data_path, 
        num_xgb_runs=num_xgb_runs, 
        experiment_name=EXPERIMENT_NAME,
        tracking_uri=MLFLOW_TRACKING_URI,
    )
    
    # Stage best model
    stage_model(
        tracking_uri=MLFLOW_TRACKING_URI, 
        experiment_name=EXPERIMENT_NAME
    )
```

Here `mlflow_staging` creates a new experiment every time it runs with name based on the scheduled start time (obtained using the run context `ctx`). It's important that injecting the correct datetime parameter (for identifying the experiment) is delegated to the orchestrator instead of the executing machine. For example, the clock on the executing machine might be rouge, in a different timezone, or the job may be queued for some reason so that execution time is delayed. This is the case for the test run in the following figure.


```{figure} ../../../img/flow-context.png
---
---
Run context for a test flow that simply logs the scheduled time. For some reason, execution is delayed so that the execution time differs from the scheduled time. Also, the orchestrator time is in UTC (standard) while the other values are in UTC+8 (system time).
```


Later we see that we can specify a parameter dictionary for passing parameter values in the flow during deployment. So it is good practice to run a flow with a test parameters dict. Thus, we append the script with the following:

```python
if __name__ == "__main__":
    
    parameters={
        "train_data_path": './data/green_tripdata_2021-01.parquet',
        "valid_data_path": './data/green_tripdata_2021-02.parquet',
        "num_xgb_runs": 3,
    }

    mlflow_staging(**parameters)
```

In [166]:
!python main.py

00:28:34.569 | INFO    | prefect.engine - Created flow run 'flashy-dove' for flow 'mlflow-staging'
00:28:34.569 | INFO    | Flow run 'flashy-dove' - Using task runner 'SequentialTaskRunner'
00:28:34.623 | INFO    | Flow run 'flashy-dove' - Created subflow run 'uppish-earthworm' for flow 'main'
2022/06/13 00:28:34 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment-2022-06-13 00:28:34.236376' does not exist. Creating a new experiment.
00:28:34.774 | INFO    | Flow run 'uppish-earthworm' - Created subflow run 'bizarre-trout' for flow 'preprocess-data'
00:28:34.823 | INFO    | Flow run 'bizarre-trout' - Created task run 'load_training_dataframe-4335dacd-0' for task 'load_training_dataframe'
00:28:34.851 | INFO    | Flow run 'bizarre-trout' - Created task run 'load_training_dataframe-4335dacd-1' for task 'load_training_dataframe'
00:28:34.875 | INFO    | Flow run 'bizarre-trout' - Created task run 'fit_preprocessor-5181a278-0' for task 'fit_preprocessor'
00:28:34.980 | I

Let's check out `flashy-dove` in the Prefect Orion UI. Looks good. Here we see the `main` subflow for training and the `stage_model` task for staging the best model in the experiment that was just performed. This can all be checked in MLflow.

```{figure} ../../../img/flashy-dove.png
---
---
```

LGTM, let's deploy!

### Local storage setup

Before creating a deployment in Prefect let us first setup a local storage for saving for persisting flow code for deployments, task results, and flow results. This is simple enough to do. Take note of the storage identifier as this will be used later.

```bash
$ prefect storage create

Found the following storage types:
0) Azure Blob Storage
    Store data in an Azure blob storage container.
1) File Storage
    Store data as a file on local or remote file systems.
2) Google Cloud Storage
    Store data in a GCS bucket.
3) Local Storage
    Store data in a run's local file system.
4) S3 Storage
    Store data in an AWS S3 bucket.
5) Temporary Local Storage
    Store data in a temporary directory in a run's local file system.
Select a storage type to create: 3
You've selected Local Storage. It has 1 option(s).
STORAGE PATH: ~/.prefect/local-storage
Choose a new name for this storage configuration: local-storage
Registered storage 'local-storage' with identifier
'33133d27-a83b-468c-bb72-a9f19bd0d157'.
```

### Deployment specification

For the deployment, we run the `mlflow-staging` flow every 5 minutes locally. We also set the parameters for each run. This adds a bit of flexibility. For now, the datasets are fixed, but it makes sense to run this on fresh data so that we actually get an updated model at each scheduled run. 

```{margin}
[`deployments.py`](https://github.com/particle1331/inefficient-networks/blob/85993956250601edeccf0d1bd5f192bb20873677/docs/notebooks/mlops/3-prefect/deployments.py)
```
```python
from prefect.deployments import DeploymentSpec
from prefect.orion.schemas.schedules import IntervalSchedule
from prefect.flow_runners import SubprocessFlowRunner
from datetime import timedelta


DeploymentSpec(
    name="deploy-mlflow-staging",
    flow_name='mlflow-staging',
    schedule=IntervalSchedule(interval=timedelta(minutes=1)),
    flow_location="./main.py",
    flow_storage="33133d27-a83b-468c-bb72-a9f19bd0d157", # local storage id
    flow_runner=SubprocessFlowRunner(),
    parameters={
        "train_data_path": './data/green_tripdata_2021-01.parquet',
        "valid_data_path": './data/green_tripdata_2021-02.parquet',
        "num_xgb_runs": 10,
    },
    tags=["ml"]
)

```

Specifying `SubprocessFlowRunner()` as flow runner, means that this flow is executed locally, e.g. not on Kubernetes or Docker containers. Here we specify the local storage identifier that we have just created above. Note that we specify the flow by name (`'mlflow-staging'`). This is why we defined `name='mlflow-staging'` in the decorator of `mlflow_staging`. To push our deployment to Prefect, we execute the following in the terminal:

In [167]:
!prefect deployment create deployments.py

Loading deployment specifications from python script at 'deployments.py'...
/Users/particle1331/miniforge3/envs/prefect/lib/python3.9/site-packages/prefect/deployments.py:247: UserWarning: You have configured local storage, this deployment will only be usable from the current machine..
  warnings.warn(
Creating deployment 'deploy-mlflow-staging' for flow 'mlflow-staging'...
Deploying flow script from '/Users/particle1331/code/inefficient-networks/docs/n
otebooks/mlops/3-prefect/main.py' using Local Storage...
Created deployment 'mlflow-staging/deploy-mlflow-staging'.
View your new deployment with: 

    prefect deployment inspect 'mlflow-staging/deploy-mlflow-staging'
Created 1 deployments!


```{figure} ../../../img/deploy-mlflow-staging.png
---
---
100 runs are now scheduled in Prefect.
```

### Adding workers to deployed workflows

Flows are now scheduled in Prefect. Notice that there are late runs. This is because we haven't attached any workers that will run these tasks. Unlike CI/CD platforms, all compute happens outside of Prefect that users will have to provide to run the scheduled workflows. So we create a **work queue** and fire up a **Prefect agent** to execute our deployment with our local compute. Note that the following setting up can also be done with the help of the UI.  

In [168]:
!prefect deployment ls

                                  Deployments                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                                 ┃ ID                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ mlflow-staging/deploy-mlflow-staging │ 271734ba-6e5e-4fa1-bf03-ab6801f7b44f │
└──────────────────────────────────────┴──────────────────────────────────────┘


In [169]:
# Copying the deployment ID above
!prefect work-queue create \
    --deployment '271734ba-6e5e-4fa1-bf03-ab6801f7b44f' \
    --flow-runner subprocess \
    mlflow-deploy-runner

UUID('3f80fde3-4390-413f-be3a-3d8da1913163')


The following shows the scheduled runs for this worker. This also confirms that we chose the correct flow runner and deployment. 

In [170]:
# Copying the worker ID above
!prefect work-queue preview 3f80fde3-4390-413f-be3a-3d8da1913163

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Scheduled Start… ┃ Run ID                 ┃ Name   ┃ Deployment ID           ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2022-06-12 16:5… │ d381605d-9ceb-4314-bb… │ woode… │ 271734ba-6e5e-4fa1-bf0… │
│ 2022-06-12 16:5… │ a7e80fc1-99ac-40f4-bb… │ brave… │ 271734ba-6e5e-4fa1-bf0… │
│ 2022-06-12 16:5… │ 93d3ad27-d7a6-48fa-bb… │ meteo… │ 271734ba-6e5e-4fa1-bf0… │
│ 2022-06-12 16:5… │ a7140abf-ffbd-4b13-b5… │ glori… │ 271734ba-6e5e-4fa1-bf0… │
│ 2022-06-12 16:5… │ 38227573-347a-4574-9c… │ denim… │ 271734ba-6e5e-4fa1-bf0… │
│ 2022-06-12 16:5… │ 95572d03-9888-436b-9b… │ arche… │ 271734ba-6e5e-4fa1-bf0… │
│ 2022-06-12 16:4… │ 8195406c-62a1-44a3-95… │ thick… │ 271734ba-6e5e-4fa1-bf0… │
│ 2022-06-12 16:4… │ bd41ec97-66e4-4061-be… │ vanil… │ 271734ba-6e5e-4fa1-bf0… │
│ 2022-06-12 16:4… │ e8427aaa-9a54-405c-95… │ nippy… │ 271734ba-6e5e-4fa1-bf0… │
│ 2022-06-12 16:4… │ 4a39721

Running the worker now, so it picks up scheduled runs.

In [172]:
# Copying the worker ID above
!prefect agent start 3f80fde3-4390-413f-be3a-3d8da1913163

Starting agent with ephemeral API...

  ___ ___ ___ ___ ___ ___ _____     _   ___ ___ _  _ _____
 | _ \ _ \ __| __| __/ __|_   _|   /_\ / __| __| \| |_   _|
 |  _/   / _|| _|| _| (__  | |    / _ \ (_ | _|| .` | | |
 |_| |_|_\___|_| |___\___| |_|   /_/ \_\___|___|_|\_| |_|


Agent started! Looking for work from queue 
'3f80fde3-4390-413f-be3a-3d8da1913163'...
00:55:53.694 | INFO    | prefect.agent - Submitting flow run 'f01c4790-54bf-464b-b6c9-59b2104d4751'
00:55:58.730 | INFO    | prefect.agent - Submitting flow run 'f01c4790-54bf-464b-b6c9-59b2104d4751'
00:55:59.751 | INFO    | prefect.flow_runner.subprocess - Opening subprocess for flow run 'f01c4790-54bf-464b-b6c9-59b2104d4751'...
00:55:59.761 | INFO    | prefect.agent - Completed submission of flow run 'f01c4790-54bf-464b-b6c9-59b2104d4751'
00:56:02.765 | INFO    | Flow run 'gleaming-honeybee' - Using task runner 'SequentialTaskRunner'
00:56:02.814 | INFO    | Flow run 'gleaming-honeybee' - Created subflow run 'wisteria-shrimp' for

We stop the run since this will take a while. But we already see that it has completed one flow, while some flows are still running, some are late, and many are scheduled. The late flows are because we had 1 minute deployments and it took a while for us to create a worker. Another nice thing to notice with the worker works concurrently on late flows.

```{figure} ../../../img/deployment-1.png
---
---
```

```{figure} ../../../img/deployment-2.png
---
---
```

From the log on `00:58:47.290` we see that the completed flow `gleaming-honeybee` created Version 4 of the model `NYCRideDurationModel`. Indeed, we have this version staged in the model registry:

In [177]:
client.get_latest_versions(name='NYCRideDurationModel')

[<ModelVersion: creation_timestamp=1655053127536, current_stage='Staging', description=('[2022-06-13 00:58:47.571025] The model version 4 from experiment '
  "'nyc-taxi-experiment-2022-06-13 00:44:59.733397' was transitioned to "
  'Staging.'), last_updated_timestamp=1655053127571, name='NYCRideDurationModel', run_id='cf5964fcf6dd47189ba6a5707360155f', run_link=None, source='./mlruns/8/cf5964fcf6dd47189ba6a5707360155f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>]